# Equity Loss Analysis for Atlanta MSA

## Data Sources
- Fulton County digest parcel data from 2011 to 2022 (selected for LUC=101, SFHs), excel
- Fulton County digest parcel data for 2022 (for geocoding), geojson
- Fulton County sales data from 2011 to 2022, txt
- Atlanta Neighborhood Statistical Areas with supplemental data from Census (), 2022, csv from Neighborhood Nexus
- Neighborhood characteristics? unknown

**Note: NSAs in DeKalb are excluded, we do not have data for all years**

Those neighborhoods are:
- Candler Park, Druid Hills
- Lake Claire
- East Lake
- Kirkwood
- Edgewood
- East Atlanta
- Emory University/Center for Disease Control
- Part of Morningside/Lenox Park

This leaves _ neighborhoods (see appendix for list)

## Areas of Analysis
- Corporate power in buying and purchasing (stat significance in purachsing price diff?)
- Corporate profits from rentals
- Corporate concentration
- Neighborhood characteristics?

- Sum of buying, selling -> all sales
- Sum of holding -> all parcels
- Create a cumulative measure and normalized by neighborhood context
- Take distribution of all sales to ind, corp and compare to see if statistically significant
- FLIPPING ACTIVITY
- Correlate to neighborhood characteristics
- Predict based on neighborhood characteristics
- Geospatial for each neighborhood
- Foreclosure rate 

In [1]:
import pandas as pd
import geopandas as gpd
import plotly.express as px

pd.set_option('display.max_columns', 150)
pd.options.display.float_format = '{:.5f}'.format

### Data from process_data.ipynb

In [2]:
# All sales in Fulton for period, LUC == 101
fulton_sales_all = pd.read_parquet("./output/fulton_sales_all.parquet")
# Parcel data for every year and parcel in the period, LUC == 101
digest_full_geo_nbhd = pd.read_parquet("./output/digest_full_geo_nbhd.parquet")

### Initial, basic data cleaning for our research question

PARCEL: ---
- Drop duplicate TAXYR, PARID keys (parcels that contain multiple structures, these are essentially ADUs since we are only looking at single-family); retain only the row with the largest living area square footage (main structure).

SALES: ---
- Only retain sales with valid saleval code (saleval=0)
- Drop sales with low sales price, indicating non-arms length transcations (handled by excluding saleval code T)

Notable Saleval codes:
- 0 = valid sale
- T = sale under $1000
- G = deed of gift
- 5 = Foreclosure
- 9 = Unvalidated/Deed stamps
- 3 = Remodeled after sale (flipping)

### Parcel data cleaning

In [3]:
# Investigate the cause of TAXYR, PARID duplicate keys
digest_full_geo_nbhd[digest_full_geo_nbhd.duplicated(subset=["TAXYR", "PARID"], keep=False)].sort_values(by=["TAXYR", "PARID"]).head(5)

,PARID,OBJECTID,geometry,TAXYR,Situs Adrno,Situs Adrdir,Situs Adrstr,Situs Adrsuf,Cityname,Luc,Calcacres,Own1,Own2,Owner Adrno,Owner Adradd,Owner Adrdir,Owner Adrstr,Owner Adrsuf,own_cityname,Statecode,own_zip,D Yrblt,D Effyr,D Yrremod,Sfla,neighborhood
1306124,06 031200010082,160985,"POLYGON ((-84.270547 33.960675, -84.270835 33....",2010,7615,,NESBIT FERRY,RD,SANDY SPRINGS,101,1.00550,MC BRIDE LAVONNE G & MICHELLE,,7615,,,NESBIT FERRY,RD,ATLANTA,GA,30350,1972,0,0,3975.00000,None
1306126,06 031200010082,160985,"POLYGON ((-84.270547 33.960675, -84.270835 33....",2010,7615,,NESBIT FERRY,RD,SANDY SPRINGS,101,1.00550,MC BRIDE LAVONNE G & MICHELLE,,7615,,,NESBIT FERRY,RD,ATLANTA,GA,30350,1974,0,0,1520.00000,None
1339237,06 031200030064,167046,"POLYGON ((-84.271076 33.962745, -84.271477 33....",2010,5020,,SPALDING,DR,SANDY SPRINGS,101,0.82540,GOLDBY FRANCES R & F SCOTT,,5020,,,SPALDING,DR,DUNWOODY,GA,30350,1973,0,0,3669.00000,None
1339238,06 031200030064,167046,"POLYGON ((-84.271076 33.962745, -84.271477 33....",2010,5020,,SPALDING,DR,SANDY SPRINGS,101,0.82540,GOLDBY FRANCES R & F SCOTT,,5020,,,SPALDING,DR,DUNWOODY,GA,30350,2002,0,0,1003.00000,None
1324623,06 0338 LL0241,165849,"POLYGON ((-84.300355 33.962623, -84.300056 33....",2010,2100,,DUNWOODY HERITAGE,DR,SANDY SPRINGS,101,1.60700,PACETTI MICHAEL K & EILEEN H,,2100,,,DUNWOODY HERITAGE,DR,DUNWOODY,GA,30350,1962,0,0,667.00000,None


TAXYR, PARID duplicate keys appear to be caused by ADUs; since we are only investigating LUC=101 (detached single-family), this makes sense. Upon confirming from Google Maps, the properties above did have ADUs. The data shows each record refers to a different structure with a different year built and square footage.

We will simply take the row with the largest square footage. First, we can see below that they are not significant in number. Second, we are only interested buying and selling activity as well as rentals by corporates. If a parcel is purchased, all structures on the parcel are purchased. While corporates can rent out ADUs, we will later use Fair Market Value to from the sales data to calculate rents, which includes the entire transcation.

In [4]:
init_len = len(digest_full_geo_nbhd)

digest_full_geo_nbhd = digest_full_geo_nbhd.sort_values(by="Sfla").drop_duplicates(subset=["TAXYR", "PARID"], keep="first")

print(f"Number of dropped duplicates: {init_len - len(digest_full_geo_nbhd)}")
print(f"Number of total parcels (all years): {len(digest_full_geo_nbhd)}")

Number of dropped duplicates: 27680
Number of total parcels (all years): 2750929


### Sale data cleaning

In [5]:
# Count of each saleval code
fulton_sales_all.groupby("Saleval")["Saleval"].count().sort_values(ascending=False).head(10)

Saleval
0     117330
T      36730
G      23159
5      17287
M      13777
9      13131
3      10205
RE      8124
4       7015
4E      6112
Name: Saleval, dtype: int64

In [6]:
# Investigating foreclosure sales as those might be of interest
fulton_sales_all[fulton_sales_all["Saleval"] == "5"].sample(5)

,TAXYR,PARID,Luc,Saledt,SALES PRICE,FAIR MARKET VALUE,DEED TYPE,Costval,Saleval,GRANTOR,GRANTEE
10627,2013,12 172202980282,101,07-FEB-2012,0.00000,290000.00000,DP,290000,5,CULDA NICOLAIE,BANK OF AMERICA N A
2495,2012,09F020200131871,101,02-AUG-2011,121293.00000,75300.00000,SW,75300,5,ONEWEST BANK FSB,FEDERAL NATIONAL MORTGAGE ASSOCIATION
16891,2013,14 000200020809,101,04-SEP-2012,24200.00000,13200.00000,FD,13200,5,MANNING EUGENE,METLIFE HOME LOANS
21443,2012,14 014300070135,101,05-JUL-2011,147309.00000,13300.00000,DP,13300,5,FULTON REGINALD L,BAC HOME LOANS SERVICING LP
6629,2011,09F370001545400,101,05-OCT-2010,75468.00000,74500.00000,FD,74500,5,ESUKE GEORGE,JPMORGAN CHASE BANK NATIONAL ASSN


In [7]:
# Cleaning sales
fulton_sales_all = fulton_sales_all[fulton_sales_all["Saleval"] == "0"]
print(f"Number of sales: {len(fulton_sales_all)}")

Number of sales: 117330


### Drop parcels and sales where government institutions or banks are owners

In [8]:
govt_keywords = ['FEDERAL'] # FANNIE AND FREDDIE MAE PUT FEDERAL IN THEIR NAMES
bank_keywords = [
    'BANK', 'MORTGAGE', 'LENDING', 'LOAN',
    'FINANCE', 'FUND', 'CREDIT', 'TRUST', 'SERVICES'
]
govt = []
banks = []

govt += fulton_sales_all[
    fulton_sales_all['GRANTEE'].apply(lambda x: any([key in str(x) for key in govt_keywords]))
]['GRANTEE'].unique().tolist() + fulton_sales_all[
    fulton_sales_all['GRANTOR'].apply(lambda x: any([key in str(x) for key in govt_keywords]))
]['GRANTOR'].unique().tolist() + digest_full_geo_nbhd[
    digest_full_geo_nbhd["Own1"].apply(lambda x: any([key in str(x) for key in govt_keywords]))
]['Own1'].unique().tolist()

banks += fulton_sales_all[
    fulton_sales_all['GRANTEE'].apply(lambda x: any([key in str(x) for key in bank_keywords]))
]['GRANTEE'].unique().tolist() + fulton_sales_all[
    fulton_sales_all['GRANTOR'].apply(lambda x: any([key in str(x) for key in bank_keywords]))
]['GRANTOR'].unique().tolist() + digest_full_geo_nbhd[
    digest_full_geo_nbhd["Own1"].apply(lambda x: any([key in str(x) for key in bank_keywords]))
]['Own1'].unique().tolist()

print("Sales")
print("Size before: ", fulton_sales_all.shape)
fulton_sales_all = fulton_sales_all[
    ~(
        fulton_sales_all['GRANTEE'].isin(govt + banks)
        | fulton_sales_all['GRANTOR'].isin(govt + banks)
    )
]
print("Size after: ", fulton_sales_all.shape)
print("")

print("Digest")
print("Size before: ", digest_full_geo_nbhd.shape)
digest_full_geo_nbhd = digest_full_geo_nbhd[
    ~(digest_full_geo_nbhd['Own1'].isin(govt + banks))
]
print("Size after: ", digest_full_geo_nbhd.shape)
print("")


Sales
Size before:  (117330, 11)
Size after:  (113325, 11)

Digest
Size before:  (2750929, 26)
Size after:  (2692761, 26)



In [9]:
# save record of entities classified as govt or bank
with open("./output/govt.txt", "w") as f:
    f.write("\n".join(govt))
with open("./output/banks.txt", "w") as f:
    f.write("\n".join(banks))

### Basic methodology to identify same owners - needed for further analysis
- Drop any rows without Owner Address
- Create an Owner Address (labeled: "owner_addr") column that is the concatentation of owner address number, owner address string, and owner zip.
- If address string contains numbers, then it is a PO BOX. However, a lot are formatted in different ways, such as P O BOX 123, PO BOX 123, P.O. BOX 123, etc. We can retain the number from the address string, and manually prepend PO BOX, so all will have an identical format.

**Why:** these values get us a highly accurate key for same owner. Owner address string does not contain postfixes like ST, AVE, etc. that might cause issues. Combined with owner number and owner zip, we can say with high confidence that the address is the same while avoiding many common differences amongst the same address (ST vs STREET, etc.). This method is prefered over names which has a higher chance of false positive, and large corporations may operate with differently named subsidaries. This method may also undercount, if a company uses multiple addresses, but this is somewhat unlikely and undercounting is simply an acceptable limitation. It is acceptable since large investors (who would use different addresses) will own so many properties with each subsidary that it will be binned in the correct bin regardless.

In [10]:
# Drop rows without an owner address
init_len = len(digest_full_geo_nbhd)
digest_full_geo_nbhd = digest_full_geo_nbhd[digest_full_geo_nbhd["Owner Adrstr"] != ""]
print(f"Number of empty addresses dropped: {init_len - len(digest_full_geo_nbhd)}")

Number of empty addresses dropped: 3462


In [11]:
# Demonstration of PO BOX issue
re_letters_then_numbers = r"^[a-zA-Z ]*[0-9]+"

digest_full_geo_nbhd[
    digest_full_geo_nbhd["Owner Adrstr"].str.contains(re_letters_then_numbers, regex=True)
]["Owner Adrstr"].sample(5)

1652722    P O BOX 391193
186023     P O BOX 888774
2692890    P O BOX 922073
121471       P O BOX 1081
1939227     P O BOX 42495
Name: Owner Adrstr, dtype: string

In [12]:
# Demonstration that if an address string is a PO BOX, it contains "BOX" and numbers
re_contains_weird_box = r".*B\.O\.X.*"
re_box_and_numbers = r".*BOX.*[0-9].*"

print(len(digest_full_geo_nbhd[
    digest_full_geo_nbhd["Owner Adrstr"].str.contains(re_contains_weird_box, regex=True)
]["Owner Adrstr"]))

digest_full_geo_nbhd[
    digest_full_geo_nbhd["Owner Adrstr"].str.contains(re_box_and_numbers, regex=True)
]["Owner Adrstr"].sample(10)

0


2355996      P O BOX 82833
104671         P O BOX 201
338475     P.O. BOX 922265
119324         P O BOX 368
1088011       P O BOX 7808
2476094      P O BOX 45402
2778193      P.O. BOX 2903
1648789      P.O. BOX 1805
206724         P O BOX 233
1921244        P.O. BOX 81
Name: Owner Adrstr, dtype: string

In [13]:
# Re-format PO BOXES
re_capture_numbers = r"([0-9]+)"
digest_full_geo_nbhd["mod_own_adrstr"] = digest_full_geo_nbhd["Owner Adrstr"].copy(deep=True)

mask = digest_full_geo_nbhd["mod_own_adrstr"].str.contains(re_box_and_numbers, regex=True)

digest_full_geo_nbhd.loc[mask, "mod_own_adrstr"] = "PO BOX " + digest_full_geo_nbhd.loc[
    mask, "mod_own_adrstr"
].str.extract(re_capture_numbers)[0]

In [14]:
# Print total number of PO BOXES without a number in their address string
re_po_box_no_number = r"^(?!.*\d)[P]+.* BOX.*"
len(digest_full_geo_nbhd[digest_full_geo_nbhd["mod_own_adrstr"].str.contains(
    re_po_box_no_number, regex=True
)][["Owner Adrno", "mod_own_adrstr"]])

37

There's not enough PO Boxes without numbers to worry about accounting for this.

In [15]:
# should use sample multiple times to find instances of PO Boxes
digest_full_geo_nbhd[["Owner Adrno", "Owner Adrstr", "mod_own_adrstr"]].sample(20)

,Owner Adrno,Owner Adrstr,mod_own_adrstr
2391409,1117,WOODSONG PASS,WOODSONG PASS
1834999,1865,WEST VESTA,WEST VESTA
231894,845,BROOKSGLEN,BROOKSGLEN
1814875,1341,LOCKWOOD,LOCKWOOD
2292305,0,P O BOX 29587,PO BOX 29587
1376994,321,PINELAND,PINELAND
1569603,931,MONROE,MONROE
835449,410,WOOD TRACE,WOOD TRACE
2446139,1423,HARTFORD,HARTFORD
2742495,2105,KENSLEY,KENSLEY


Appears to work as expected.

In [16]:
# Regex to clean by replacing dots, commas, and multiple spaces
# Also make all strings uppercase (they should be already)

re_dots_commas = r"[.,]+"
re_multiple_spaces = r"\s{2,}"

digest_full_geo_nbhd["owner_addr"] = (
    digest_full_geo_nbhd["Owner Adrno"].astype(str) + " " +
    digest_full_geo_nbhd["mod_own_adrstr"] + " " +
    digest_full_geo_nbhd["own_zip"]
).str.replace(
    re_dots_commas,
    "",
    regex=True
).str.replace(
    re_multiple_spaces,
    " ",
    regex=True
).str.upper()

In [17]:
# Lets validate the accuracy of this approach
digest_full_geo_nbhd.groupby(
    "owner_addr"
).agg(
    {
        "Own1": lambda x: list(x),
        "owner_addr": "count"
    }
).rename(
    columns={
        "owner_addr": "count"
    }
).sort_values(
    by="count",
    ascending=False
).head(5)

,Own1,count
owner_addr,,
3505 KOGER 30096,"[RNTR 3 LLC, RNTR 3 LLC, FYR SFR BORROWER LLC,...",2493
5001 PLAZA ON THE 78746,"[ALTO ASSET COMPANY 2 LLC, ALTO ASSET COMPANY ...",2289
1717 MAIN 75201,"[2018 3 IH BORROWER LP, 2018 3 IH BORROWER LP,...",2022
901 MAIN 75202,"[2015 3 IH2 BORROWER LP, 2015 3 IH2 BORROWER L...",1593
4400 WILL ROGERS 73108,"[SECRETARY OF HOUSING & URBAN DEV, SECRETARY O...",1518


Full method (creating a cleaned owner_addr column to aggreggate on) also appears to work as expected.

### Determine the scale of ownership for each parcel owner and year at the neighorhood, city, and county level; create an ownership table
E.g. each parcel will have a column with a sum and percent of properties owned by the parcel owner in the given neighborhood, ATL, and in Fulton county for that TAXYR.

Later we can put these into discrete bins if needed.

In [18]:

# Caculate number, percent of parcels owned in all of Fulton in each year

fulton_parcel_count_yr = pd.DataFrame(
    digest_full_geo_nbhd.groupby("TAXYR")["PARID"].count()
).rename(columns={
    "PARID": "fulton_parcels_taxyr"
})

all_fulton = digest_full_geo_nbhd.groupby(
    ["TAXYR", "owner_addr"]
).agg(
    {"owner_addr": "count"}
).rename(
    columns={"owner_addr": "count_owned_fulton"}
).reset_index().merge(
    fulton_parcel_count_yr,
    on="TAXYR",
    how="inner"
)

all_fulton["pct_owned_fulton"] = all_fulton["count_owned_fulton"] / all_fulton["fulton_parcels_taxyr"] * 100

# Caculate number, percent of parcels owned in ATL in each year
# TODO am I dropping some because Im doing an inner merge?
atl_parcels_only = digest_full_geo_nbhd[digest_full_geo_nbhd["neighborhood"].notna()]
atl_parcel_count_yr = pd.DataFrame(
    atl_parcels_only.groupby("TAXYR")["PARID"].count()
).rename(columns={
    "PARID": "atl_parcels_taxyr"
})

all_atl = atl_parcels_only.groupby(
    ["TAXYR", "owner_addr"]
).agg(
    {"owner_addr": "count"}
).rename(
    columns={"owner_addr": "count_owned_atl"}
).reset_index().merge(
    atl_parcel_count_yr,
    on="TAXYR",
    how="inner"
)

all_atl["pct_owned_atl"] = all_atl["count_owned_atl"] / all_atl["atl_parcels_taxyr"] * 100

# Caculate number, percent of parcels owned in the parcel's neighorbohood in each year

nbhd_parcel_count_yr = pd.DataFrame(
    atl_parcels_only.groupby(
        ["neighborhood", "TAXYR"]
    )["PARID"].count()
).rename(columns={"PARID": "neighborhood_parcels_taxyr"}) #.reset_index()

all_neighborhood = atl_parcels_only.groupby(
    ["TAXYR", "owner_addr", "neighborhood"]
).agg(
    {"owner_addr": "count"}
).rename(
    columns={"owner_addr": "count_owned_neighborhood"}
).reset_index().merge(
    nbhd_parcel_count_yr,
    on=["TAXYR", "neighborhood"],
    how="inner"
)

all_neighborhood["pct_owned_neighborhood"] = all_neighborhood[
    "count_owned_neighborhood"
] / all_neighborhood[
    "neighborhood_parcels_taxyr"
] * 100

all_ownership_levels = all_fulton.merge(
    all_atl, on=["TAXYR", "owner_addr"], how="inner"
).merge(
    all_neighborhood, on=["TAXYR", "owner_addr"], how="outer"
)

Now we have a table that lists each owner and their concentration ownership concentration for every TAXYR in each neighborhood, in Atlanta, and in Fulton. Note: this table only includes neighborhoods where that owner has at least one property.

In [19]:
all_ownership_levels.sort_values(by="pct_owned_fulton", ascending=False).head(5)

,TAXYR,owner_addr,count_owned_fulton,fulton_parcels_taxyr,pct_owned_fulton,count_owned_atl,atl_parcels_taxyr,pct_owned_atl,neighborhood,count_owned_neighborhood,neighborhood_parcels_taxyr,pct_owned_neighborhood
886300,2022,5001 PLAZA ON THE 78746,714,216467,0.32984,101,75679,0.13346,South River Gardens,4,596,0.67114
886316,2022,5001 PLAZA ON THE 78746,714,216467,0.32984,101,75679,0.13346,"Fairburn Mays, Mays",1,250,0.40000
886301,2022,5001 PLAZA ON THE 78746,714,216467,0.32984,101,75679,0.13346,Cascade Avenue/Road,1,835,0.11976
886302,2022,5001 PLAZA ON THE 78746,714,216467,0.32984,101,75679,0.13346,Grove Park,3,1623,0.18484
886303,2022,5001 PLAZA ON THE 78746,714,216467,0.32984,101,75679,0.13346,"Arlington Estates, Ben Hill, Butner/Tell, Elmc...",7,1229,0.56957


In [20]:
# Aggregate all names from parcel data that have been matched to the same address; e.g. make a list of owner names for an owner address
same_owners = pd.DataFrame(digest_full_geo_nbhd.groupby("owner_addr")["Own1"].unique().apply(lambda x: ' - '.join(x))).merge(
    all_ownership_levels,
    on="owner_addr",
    how="inner"
).rename(columns={"Own1": "all_assoc_names"}).sort_values(by="pct_owned_fulton", ascending=False)[
    [
        'owner_addr', 'TAXYR', 'neighborhood', 'count_owned_neighborhood', 'neighborhood_parcels_taxyr', 'pct_owned_neighborhood',
        'count_owned_atl', 'atl_parcels_taxyr', 'pct_owned_atl', 'count_owned_fulton', 'fulton_parcels_taxyr', 'pct_owned_fulton', 'all_assoc_names'
    ]
]

In [21]:
# TODO make a ownership table agg'd for all YEARS and save separately

In [22]:
# Save output
all_ownership_levels.to_csv("./output/all_ownership_levels.csv", index=False)
same_owners.to_csv("./output/same_owners.csv", index=False)

### Determine the scale of buying and selling activity for each owner at the neighborhood, city, and county level; create a sales activity table
E.g. each parcel will have a column with a aggregated sum and percent of properties purchased, sold, and overall activity by the parcel owner in the given neighborhood, ATL, and in Fulton county for that TAXYR.

Later we can put these into discrete bins if needed.

**Method**:
- Sales data does not contain buyer or seller address. We can't simply use GRANTEE or GRANTOR name, because names can be different for the same owner corporation (subsidaries, typos). Instead we identify buyer and seller address by:
    - Match GRANTEE name to parcel data on GRANTEE = Own1 (owner name) and extract owner_addr for CURRENT TAXYR
    - Match GRANTOR name to parcel data on GRANTOR = Own1 (owner name) and extract owner_addr for PREVIOUS TAXYR
    - For names where the GRANTEE or GRANTOR name doesn't match exactly (due to typos, etc.), we can take the owner_addr with the same method ONLY IF there was only one sale in the given TAXYR. In the case of multiple sales in one TAXYR, the last purchaser appears to be recorded in the parcel data as the owner (see evidence below); if we tried to match an earlier sale in that year, we would get the wrong owner address. This is a problem because we want the purchaser address for each sale to appropriately account for flipping activity for example.
    - Else, try to find an exact owner name match from all parcel data, not limited to PARID and TAXYR; use the first match if a match is found. Then try to match owner name with GA Business Registry data. Although this won't account for individuals, but its unlikely individuals would be involved in multiple transcations on same property in one year, and we don't care much about individuals. An individual without a corporation will almost definitely not have the capital to be doing this for many properties.
    - See below for verifcation that the sales and parcel data can be correctly matched this way
- In short:
    - Try to match by owner name, PARID, and TAXYR
    - If no match, get match from just PARID and TAXYR, ONLY IF there is a single transcation in the given TAXYR for that PARID
    - Else, try to find an exact owner name match from all parcel data, not limited to PARID and TAXYR; use the last match if a match is found (last because that is most recent address of company)
    - Maybe try GA Business REGISTRY -----
    - Where none of the above methods work, drop if total count is insignificant
- Aggregate sales for each year by their owner address, identify the number of purchase, sell, and total transcations of that owner in the given TAXYR.

In [23]:
# Minor cleaning on GRANTEE, GRANTOR, and Own1 (parcel data)
# Regex to clean by replacing dots, commas, and multiple spaces
# Also make all strings uppercase (they should be already)
re_dots_commas = r"[.,]+"
re_multiple_spaces = r"\s{2,}"

for col in ["GRANTEE", "GRANTOR"]:
    fulton_sales_all[col] = fulton_sales_all[col].str.replace(
        re_dots_commas, "", regex=True
    ).str.replace(
        re_multiple_spaces, " ", regex=True
    ).str.upper()
    
digest_full_geo_nbhd["Own1"] = digest_full_geo_nbhd["Own1"].str.replace(
    re_dots_commas, "", regex=True
).str.replace(
    re_multiple_spaces, " ", regex=True
).str.upper()

In [24]:
# Verify that GRANTEE = Own1 in TAXYR, GRANTOR = Own1 in PREVIOUS TAXYR
# where GRANTEE/GRANTOR from sale data, Own1 from parcel data.
# Take a random PARID with at least one sale, pull up its sales and parcel data, then compare

# Note: first time this was ran, sample was PARID="14 012400100182"
# fulton_sales_all.sample(1)["PARID"]
fulton_sales_all[fulton_sales_all["PARID"] == "14 012400100182"][["TAXYR", "PARID", "GRANTOR", "GRANTEE"]].sort_values(by="TAXYR")

,TAXYR,PARID,GRANTOR,GRANTEE
29610,2022,14 012400100182,DANLEY DEVELOPMENT GROUP INC,SCHNEIDER KRISTIN ANNE &


In [25]:
digest_full_geo_nbhd[digest_full_geo_nbhd["PARID"] == "14 012400100182"][["TAXYR", "PARID", "Own1"]].sort_values(by="TAXYR")

,TAXYR,PARID,Own1
2469107,2010,14 012400100182,BANNISTER EARLENE V
2469106,2011,14 012400100182,BANNISTER EARLENE V
2469108,2012,14 012400100182,BANNISTER EARLENE V
2469109,2013,14 012400100182,BANNISTER EARLENE V
2469110,2014,14 012400100182,BANNISTER EARLENE V
2469104,2015,14 012400100182,BANNISTER EARLENE V
2469105,2017,14 012400100182,BANNISTER EARLENE V
2469111,2018,14 012400100182,HERSHBERGER JAMES
2469112,2019,14 012400100182,GREEN ENERGY LIGHTING LLC
2469113,2020,14 012400100182,DANLEY DEVELOPMENT GROUP INC


In [26]:
count_sales_yr = pd.DataFrame(
    fulton_sales_all.groupby(["TAXYR", "PARID"])["PARID"].count()
).rename(columns={"PARID": "count_sales_yr"})

fulton_sales_all = fulton_sales_all.merge(
    count_sales_yr,
    on=["TAXYR", "PARID"],
    how="inner"
)

more_than_one_sale_yr = len(
    fulton_sales_all[fulton_sales_all["count_sales_yr"] > 1].drop_duplicates(
        subset=["TAXYR", "PARID"]
    )
)
print(f"Count of properties that sold multiple times in one year: {more_than_one_sale_yr}")
count_sales_yr.sort_values(by="count_sales_yr", ascending=False).head(5)

Count of properties that sold multiple times in one year: 1177


,,count_sales_yr
TAXYR,PARID,
2014,14 008900040456,4
2013,14 003900070074,4
2014,14 003500030759,4
2011,14 016300160964,4
2015,14 015900040097,4


In [27]:
# Identify matches for GRANTEE, GRANTOR, and Own1 (parcel data)
digest_df = digest_full_geo_nbhd[['PARID', 'TAXYR', 'owner_addr', 'Own1']].copy(deep=True)
for person in ["GRANTEE", "GRANTOR"]:
    if person == "GRANTOR":
        digest_df["TAXYR"] = digest_df["TAXYR"] + 1
    matches = {
        "exact": {"left": ['PARID', 'TAXYR', person], "right": ['PARID', 'TAXYR', 'Own1']},
        "single_sale": {"left": ['PARID', 'TAXYR'], "right": ['PARID', 'TAXYR']},
        "only_exact_name": {"left": [person], "right": ['Own1']}
    }
    for match in matches:
    
        df = fulton_sales_all
        if match == "single_sale":
            df = df[df["count_sales_yr"] == 1]
            matched_df = df.merge(
                digest_df[['PARID', 'TAXYR', 'owner_addr', 'Own1']],
                on=matches[match]["left"],
                how='inner'
            )
        elif match == "only_exact_name":
            matched_df = df.merge(
                digest_df.drop_duplicates(subset=["Own1"], keep="last")[['Own1', 'owner_addr']],
                left_on=matches[match]["left"],
                right_on=matches[match]["right"],
                how='inner'
            )
        else:
            matched_df = df.merge(
                digest_df[['PARID', 'TAXYR', 'owner_addr', 'Own1']],
                left_on=matches[match]["left"],
                right_on=matches[match]["right"],
                how='inner'
            )
        
        if match == "only_exact_name":
            fulton_sales_all = fulton_sales_all.merge(
                matched_df[["Own1", "owner_addr"]].drop_duplicates(),
                left_on=matches[match]["left"],
                right_on=matches[match]["right"],
                how="left"
            ).rename(
                columns={"Own1": f"{person}_{match}", "owner_addr": f"{person}_{match}_addr"}
            )
        else:
            fulton_sales_all = fulton_sales_all.merge(
                matched_df[["TAXYR", "PARID", "Own1", "owner_addr"]],
                on=["TAXYR", "PARID"],
                how="left"
            ).rename(
                columns={"Own1": f"{person}_{match}", "owner_addr": f"{person}_{match}_addr"}
            )
        
        display(fulton_sales_all[["TAXYR", "PARID", f"{person}", f"{person}_{match}"]].sample(5))

,TAXYR,PARID,GRANTEE,GRANTEE_exact
55990,2018,11 096300321994,KIM CHANG SOO,KIM CHANG SOO
111014,2022,17 015400070597,MORAN SETH DANIEL &,MORAN SETH DANIEL &
18455,2014,09F220100991388,LANDSMITH LP,AMERICAN RESIDENTIAL LEASING COMPANY LLC
70400,2019,14 015200140159,RINGHAM ANDREW,RINGHAM ANDREW
105894,2022,14 004300050039,BARTOSIAK DESIREE MARIE,BARTOSIAK DESIREE MARIE


,TAXYR,PARID,GRANTEE,GRANTEE_single_sale
22541,2014,14 008700061280,FLOWMAX HOLDINGS LLC,<NA>
60344,2018,14F0158 LL3521,CERBERUS SFR HOLDINGS LP,CERBERUS SFR HOLDINGS LP
12053,2013,12 188003901129,EVERIDGE DEBBIE K,EVERIDGE DEBBIE K
69072,2019,14 001200100294,GELHAUSEN RICHARD & MICHAEL PHILLIP,GELHAUSEN RICHARD & MICHAEL PHILLIP
24197,2014,14F0099 LL1220,FRAZIER WILLIAMS DONNA M,FRAZIER WILLIAMS DONNA M


,TAXYR,PARID,GRANTEE,GRANTEE_only_exact_name
79363,2020,12 276207360083,URREA JACOB J & DENSMORE MALLORY A,URREA JACOB J & DENSMORE MALLORY A
91921,2021,12 318108940025,ROSSI JON,ROSSI JON
6922,2012,14 001400040142,STOWELL KENDALL C & NICOLAS G,STOWELL KENDALL C & NICOLAS G
26390,2014,17 0216 LL1060,PERKINS MICHELLE A,PERKINS MICHELLE A
650,2011,11 014400850838,KAMALAKAR ASHUTOSH G,KAMALAKAR ASHUTOSH G


,TAXYR,PARID,GRANTOR,GRANTOR_exact
91580,2021,12 266007111232,GARY NATTRASS AND GINA M NATTRASS,<NA>
60318,2018,14F0124 LL1559,PRUDENTIAL CONSTRUCTION INC,<NA>
35717,2015,17 006900030389,LUCOFF JUSTIN D,<NA>
21890,2014,14 002100070579,BURK JOHN B,<NA>
65330,2019,09F200000954876,CENTURY COMMUNITIES OF GEORGIA LLC,<NA>


,TAXYR,PARID,GRANTOR,GRANTOR_single_sale
109361,2022,17 0005 LL2261,JENNIFER LYNN VANCE AND DOMINI LASECKE,VANCE JENNIFER LYNN & LASECKE DOMINI
4457,2011,17 016400030052,NEMEROFF CHARLES B,NEMEROFF CHARLES B & GAYLE
51620,2017,17 0198 LL0452,KATES THOMAS W JR & ELIZABETH E,KATES THOMAS W JR
64593,2019,07 270001693609,KOTHAWALA PRAMILA K,KOTHAWALA PRAMILA K
39738,2016,12 198304480012,ZAGLIN TERESSA,ZAGLIN TERESSA


,TAXYR,PARID,GRANTOR,GRANTOR_only_exact_name
106057,2022,14 004400041268,CHEROKEE PLACE PROPERTIES LLC,CHEROKEE PLACE PROPERTIES LLC
24264,2014,14F0079 LL1190,JEFFERSON HOMES INC,JEFFERSON HOMES INC
49957,2017,17 001300010255,F & M INVESTMENTS LLC,<NA>
37109,2015,17 025300110260,TIF-GAI LLC,<NA>
32020,2015,13 016300030110,EI HOLDINGS LLC,<NA>


In [28]:
for person in ["GRANTEE", "GRANTOR"]:
    print(f"Person: {person} ---")
    fulton_sales_all[f"{person}_match"] = fulton_sales_all[f"{person}_exact"]
    fulton_sales_all[f"{person}_match_addr"] = fulton_sales_all[f"{person}_exact_addr"]
    num_matched = len(fulton_sales_all[fulton_sales_all[f'{person}_match'].notna()])
    print(f"Number exact matched: {num_matched}")
    print(f"Pct exact matched: {num_matched / len(fulton_sales_all)}")
    print("")
    
    for match in ["single_sale", "only_exact_name"]:
        fulton_sales_all[f"{person}_match"] = fulton_sales_all[f"{person}_match"].fillna(
            fulton_sales_all[f"{person}_{match}"]
        )
        fulton_sales_all[f"{person}_match_addr"] = fulton_sales_all[f"{person}_match_addr"].fillna(
            fulton_sales_all[f"{person}_{match}_addr"]
        )
        prev_matched = num_matched
        num_matched = len(fulton_sales_all[fulton_sales_all[f'{person}_match'].notna()])
        print(f"Number of additional matches with {match}: {num_matched - prev_matched}")
        print(f"Number prev matches + {match} matched: {num_matched}")
        print(f"Pct prev matches + {match} matched: {num_matched / len(fulton_sales_all)}")
        print("")
    
    print("")
    print("")

Person: GRANTEE ---
Number exact matched: 101370
Pct exact matched: 0.8927108927108927

Number of additional matches with single_sale: 10960
Number prev matches + single_sale matched: 112330
Pct prev matches + single_sale matched: 0.9892296989071183

Number of additional matches with only_exact_name: 430
Number prev matches + only_exact_name matched: 112760
Pct prev matches + only_exact_name matched: 0.9930164768874447



Person: GRANTOR ---
Number exact matched: 44225
Pct exact matched: 0.3894657120463572

Number of additional matches with single_sale: 54605
Number prev matches + single_sale matched: 98830
Pct prev matches + single_sale matched: 0.8703424832457091

Number of additional matches with only_exact_name: 10150
Number prev matches + only_exact_name matched: 108980
Pct prev matches + only_exact_name matched: 0.95972805650225





Let's briefly investigate and see if the low percent of exact GRANTOR matches is problematic.

In [29]:
fulton_sales_all.sample(15)[["TAXYR", "PARID", "GRANTOR", "GRANTOR_match"]].sort_values(by="TAXYR")

,TAXYR,PARID,GRANTOR,GRANTOR_match
1627,2011,12 266007110689,LOT ONE HOMES INC,LOT ONE HOMES INC
10032,2012,22 478009810195,AVITAN SHERRIE M,AVITAN SHERRIE M
13625,2013,14 006700010653,IM JENNY H,IM JENNY H
13087,2013,14 001100130300,GUIDED STRATEGIES LLC,<NA>
17673,2013,22 541009022531,MONTERO HECTOR L,MONTERO HECTOR L &
14820,2013,14F0122 LL2013,VENTURE HOMES INC,VENTURE HOMES INC
45984,2017,12 303308410763,DANIELLE SANDERS AND KEVIN SANDERS,SANDERS KEVIN &
48239,2017,14 013400070375,SMITH R WILSON & SALLY M,SMITH R WILSON & SALLY M
61800,2018,17 013700011288,HULL RONALD R,HULL RONALD R
60285,2018,14F0115 LL0545,HOSANG TENEKA,HOSANG TENEKA


**Investigate PARID = "14 015200120227" for 2017 where GRANTOR = KINGDOM REALTY LLC and GRANTOR_match = HUNTER TROY H JR**

In [30]:
def check_parid(parid: str):
    # Where did we get the match from?
    print("Sales data with match info")
    display(fulton_sales_all[fulton_sales_all["PARID"] == parid][
        [
            "TAXYR", "PARID", "Saledt", "GRANTEE", "GRANTOR", "GRANTOR_match", "GRANTOR_exact",
            "GRANTOR_single_sale", "GRANTOR_only_exact_name", "GRANTOR_match_addr", "SALES PRICE"
        ]
    ].sort_values(by="TAXYR"))
    print("")
    print("Digest data with parcel info")
    display(digest_full_geo_nbhd[digest_full_geo_nbhd["PARID"] == parid][
        [
            "PARID", "TAXYR", "Own1", "Own2", "owner_addr"
        ]
    ].sort_values(by="TAXYR"))

In [31]:
check_parid("14 015200120227")

Sales data with match info


,TAXYR,PARID,Saledt,GRANTEE,GRANTOR,GRANTOR_match,GRANTOR_exact,GRANTOR_single_sale,GRANTOR_only_exact_name,GRANTOR_match_addr,SALES PRICE
48536,2017,14 015200120227,16-JUN-2016,DIVINE DREAM HOMES LLC,KINGDOM REALTY LLC,HUNTER TROY H JR,<NA>,HUNTER TROY H JR,<NA>,1306 LOCKHAVEN 30311,39000.00000



Digest data with parcel info


,PARID,TAXYR,Own1,Own2,owner_addr
1818625,14 015200120227,2010,HUNTER TROY H JR,,1306 LOCKHAVEN 30311
1818626,14 015200120227,2011,HUNTER TROY H JR,,1306 LOCKHAVEN 30311
1818627,14 015200120227,2012,HUNTER TROY H JR,,1306 LOCKHAVEN 30311
1818628,14 015200120227,2013,HUNTER TROY H JR,,1306 LOCKHAVEN 30311
1818629,14 015200120227,2014,HUNTER TROY H JR,,1306 LOCKHAVEN 30311
1818623,14 015200120227,2015,HUNTER TROY H JR,,1306 LOCKHAVEN 30311
1818635,14 015200120227,2016,HUNTER TROY H JR,,1306 LOCKHAVEN 30311
1818624,14 015200120227,2017,DIVINE DREAM HOMES LLC,,2345 CAREY 30315
1818630,14 015200120227,2018,DIVINE DREAM HOMES LLC,,2345 CAREY 30315
1818631,14 015200120227,2019,DIVINE DREAM HOMES LLC,,2345 CAREY 30315


KINGDOM REALTY LLC website: "With our proprietary marketing systems, we find the best properties in foreclosure, bank owned foreclosures, Metro Atlanta investment properties for sale, handyman deals, fixer uppers, discount homes, distressed property, and buy them at great win-win prices for both us and the home seller."

**Investigate PARID = "14 007500040379" for 2019 where GRANTOR = PEACHTREE ASSET MANAGEMENT LLC and GRANTOR_match = ATL 700 800 BLOCK HOLDINGS LLC**

In [32]:
check_parid("14 007500040379")

Sales data with match info


,TAXYR,PARID,Saledt,GRANTEE,GRANTOR,GRANTOR_match,GRANTOR_exact,GRANTOR_single_sale,GRANTOR_only_exact_name,GRANTOR_match_addr,SALES PRICE
69812,2019,14 007500040379,12-OCT-2018,SUTIC MILJAN,PEACHTREE ASSET MANAGEMENT LLC,ATL 700 800 BLOCK HOLDINGS LLC,<NA>,ATL 700 800 BLOCK HOLDINGS LLC,PEACHTREE ASSET MANAGEMENT LLC,2203 CUMBERLAND 30339,235000.00000
93636,2021,14 007500040379,24-JUL-2020,WALKER LAURIE,SUTIC MILJAN,SUTIC MILJAN,SUTIC MILJAN,SUTIC MILJAN,SUTIC MILJAN,581 FORMWALT 30312,330000.00000



Digest data with parcel info


,PARID,TAXYR,Own1,Own2,owner_addr
2774987,14 007500040379,2010,CMB HOMES LLC,,117 OAKWIND POINTE 30101
2774986,14 007500040379,2011,ATL 700 800 BLOCK HOLDINGS LLC,,117 OAKWIND POINTE 30101
2774988,14 007500040379,2012,ATL 700 800 BLOCK HOLDINGS LLC,,117 OAKWIND POINTE 30101
2774989,14 007500040379,2013,ATL 700 800 BLOCK HOLDINGS LLC,,5033 COLCHESTER 30080
2774990,14 007500040379,2014,ATL 700 800 BLOCK HOLDINGS LLC,,2203 CUMBERLAND 30339
2774984,14 007500040379,2015,ATL 700 800 BLOCK HOLDINGS LLC,,2203 CUMBERLAND 30339
2774996,14 007500040379,2016,ATL 700 800 BLOCK HOLDINGS LLC,,2203 CUMBERLAND 30339
2774985,14 007500040379,2017,ATL 700 800 BLOCK HOLDINGS LLC,,2203 CUMBERLAND 30339
2774991,14 007500040379,2018,ATL 700 800 BLOCK HOLDINGS LLC,,2203 CUMBERLAND 30339
2774992,14 007500040379,2019,SUTIC MILJAN,,581 FORMWALT 30312


In [33]:
digest_full_geo_nbhd[digest_full_geo_nbhd["Own1"] == "PEACHTREE ASSET MANAGEMENT LLC"][["PARID", "TAXYR", "Own1", "Own2", "owner_addr"]].sample(5).sort_values(by="TAXYR")

,PARID,TAXYR,Own1,Own2,owner_addr
2330902,14 005600030209,2017,PEACHTREE ASSET MANAGEMENT LLC,,2203 CUMBERLAND 30339
2377596,14 007500040213,2018,PEACHTREE ASSET MANAGEMENT LLC,,2203 CUMBERLAND 30339
2330908,14 005600030209,2018,PEACHTREE ASSET MANAGEMENT LLC,,2203 CUMBERLAND 30339
2380630,14 008600031201,2018,PEACHTREE ASSET MANAGEMENT LLC,,5033 COLCHESTER 30080
2377093,14 007500010950,2019,PEACHTREE ASSET MANAGEMENT LLC,,697 COOPER 30315


PEACHTREE ASSET MANAGEMENT LLC uses two addresses; ATL 700 800 BLOCK HOLDINGS LLC uses the same two addresses. They can be treated as the same entity based on address, and are by this methodology.

**Investigate PARID = "11 108003863303" for 2022 where GRANTOR = TPG HOMES AT BELLMORE LLC	and GRANTOR_match = JOHNS CREEK 206 LLC**

In [34]:
check_parid("11 108003863303")

Sales data with match info


,TAXYR,PARID,Saledt,GRANTEE,GRANTOR,GRANTOR_match,GRANTOR_exact,GRANTOR_single_sale,GRANTOR_only_exact_name,GRANTOR_match_addr,SALES PRICE
103206,2022,11 108003863303,29-JUN-2021,MOGAL GHOUSE BAIG,TPG HOMES AT BELLMORE LLC,JOHNS CREEK 206 LLC,<NA>,JOHNS CREEK 206 LLC,TPG HOMES AT BELLMORE LLC,3131 HARVARD 75205,517299.00000



Digest data with parcel info


,PARID,TAXYR,Own1,Own2,owner_addr
2771936,11 108003863303,2021,JOHNS CREEK 206 LLC,,3131 HARVARD 75205
2771937,11 108003863303,2022,MOGAL GHOUSE BAIG,,1447 CALVERT 30097


JOHNS CREEK 206 LLC owned the property but TPG HOMES AT BELLMORE LLC carried out the transcation

In [35]:
digest_full_geo_nbhd[digest_full_geo_nbhd["Own1"] == "TPG HOMES AT BELLMORE LLC"][["PARID", "TAXYR", "Own1", "Own2", "owner_addr"]].sort_values(by="TAXYR")

,PARID,TAXYR,Own1,Own2,owner_addr
2741742,11 108003952064,2019,TPG HOMES AT BELLMORE LLC,,11340 LAKEFIELD 30097
2758795,11 108003852793,2019,TPG HOMES AT BELLMORE LLC,,11340 LAKEFIELD 30097


In [36]:
fulton_sales_all[fulton_sales_all["GRANTOR_match_addr"] == "11340 LAKEFIELD 30097"][["TAXYR", "PARID", "GRANTOR", "GRANTOR_match", "GRANTOR_match_addr"]].sample(5).sort_values(by="TAXYR")

,TAXYR,PARID,GRANTOR,GRANTOR_match,GRANTOR_match_addr
39598,2016,11 114004080582,TPG HOMES AT BELLMOORE LLC,TPG HOMES AT BELLMOORE LLC,11340 LAKEFIELD 30097
52879,2017,22 432011011961,THE PROVIDENCE GROUP OF GEORGIA CUSTOM H,THE PROVIDENCE GROUP OF GEORGIA CUSTOM H,11340 LAKEFIELD 30097
75138,2019,22 498412680878,THE PROVIDENCE GROUP OF GEORGIA CUSTOM,THE PROVIDENCE GROUP OF GEORGIA CUSTOM,11340 LAKEFIELD 30097
78360,2020,11 108003952189,TPG HOMES AT BELLMOORE LLC,TPG HOMES AT BELLMOORE LLC,11340 LAKEFIELD 30097
78332,2020,11 108003852769,TPG HOMES AT BELLMOORE LLC,TPG HOMES AT BELLMOORE LLC,11340 LAKEFIELD 30097


TPG HOMES AT BELLMORE LLC has many different entities being picked up together as the same address; the case with JOHNS CREEK 206 LLC is a bit strange

In [37]:
digest_full_geo_nbhd[digest_full_geo_nbhd["Own1"] == "JOHNS CREEK 206 LLC"][["PARID", "TAXYR", "Own1", "Own2", "owner_addr"]].sample(5).sort_values(by="TAXYR")

,PARID,TAXYR,Own1,Own2,owner_addr
2724940,11 114004080616,2016,JOHNS CREEK 206 LLC,,3131 HARVARD 75205
2739737,11 114004080806,2019,JOHNS CREEK 206 LLC,,3131 HARVARD 75205
2758738,11 108003852413,2019,JOHNS CREEK 206 LLC,,3131 HARVARD 75205
2771936,11 108003863303,2021,JOHNS CREEK 206 LLC,,3131 HARVARD 75205
2771990,11 108003853361,2021,JOHNS CREEK 206 LLC,,3131 HARVARD 75205


In [38]:
check_parid("11 108003852546")

Sales data with match info


,TAXYR,PARID,Saledt,GRANTEE,GRANTOR,GRANTOR_match,GRANTOR_exact,GRANTOR_single_sale,GRANTOR_only_exact_name,GRANTOR_match_addr,SALES PRICE
78326,2020,11 108003852546,26-JUN-2019,SPARKS NICOLE,TPG HOMES AT BELLMOORE LLC,JOHNS CREEK 206 LLC,<NA>,JOHNS CREEK 206 LLC,TPG HOMES AT BELLMOORE LLC,3131 HARVARD 75205,627692.00000



Digest data with parcel info


,PARID,TAXYR,Own1,Own2,owner_addr
2758930,11 108003852546,2019,JOHNS CREEK 206 LLC,,3131 HARVARD 75205
2758931,11 108003852546,2020,SPARKS NICOLE & TIMOTHY,,1189 HANNAFORD 30097
2758932,11 108003852546,2021,SPARKS NICOLE & TIMOTHY,,1189 HANNAFORD 30097
2758933,11 108003852546,2022,SPARKS NICOLE & TIMOTHY,,1189 HANNAFORD 30097


Randomly pulling up another parcel owned by JOHNS CREEK 206 LLC, we can see it was sold to TPG HOMES AT BELLMORE LLC; these companies likely have some sort of uncaptured relationship.

**Investigate PARID = "17 004200030217" for 2010 where GRANTOR = GAKSTATTER FRED VOLKER	and GRANTOR_match = WALZER HELEN S**

In [39]:
check_parid("17 009800030467")

Sales data with match info


,TAXYR,PARID,Saledt,GRANTEE,GRANTOR,GRANTOR_match,GRANTOR_exact,GRANTOR_single_sale,GRANTOR_only_exact_name,GRANTOR_match_addr,SALES PRICE
4064,2011,17 009800030467,30-SEP-2010,NELSEN MATTHEW S,GAKSTATTER FRED VOLKER,WALZER HELEN S,<NA>,WALZER HELEN S,<NA>,2768 BRIDLE RIDGE 30519,252000.00000
50700,2017,17 009800030467,04-OCT-2016,KADAVIL JOE &,NELSEN MATTHEW S,NELSEN MATTHEW S,NELSEN MATTHEW S,NELSEN MATTHEW S,NELSEN MATTHEW S,37 LAKELAND 30305,516000.00000



Digest data with parcel info


,PARID,TAXYR,Own1,Own2,owner_addr
1133725,17 009800030467,2010,WALZER HELEN S,,2768 BRIDLE RIDGE 30519
1133726,17 009800030467,2011,NELSEN MATTHEW S,NELSEN HEATHER M,37 LAKELAND 30305
1133727,17 009800030467,2012,NELSEN MATTHEW S,NELSEN HEATHER M,37 LAKELAND 30305
1133728,17 009800030467,2013,NELSEN MATTHEW S,NELSEN HEATHER M,37 LAKELAND 30305
1133729,17 009800030467,2014,NELSEN MATTHEW S,NELSEN HEATHER M,37 LAKELAND 30305
1133730,17 009800030467,2015,NELSEN MATTHEW S,NELSEN HEATHER M,37 LAKELAND 30305
1133731,17 009800030467,2016,NELSEN MATTHEW S,NELSEN HEATHER M,37 LAKELAND 30305
1133732,17 009800030467,2017,KADAVIL JOE &,KADAVIL ANU,37 LAKELAND 30305
1133723,17 009800030467,2018,KADAVIL JOE &,KADAVIL ANU,37 LAKELAND 30305
1133724,17 009800030467,2019,KADAVIL JOE &,KADAVIL ANU,418 COLONY LAKE ESTATES 77477


Looks like multiple individuals owned the property but only one sold (divorce etc. can explain, such property transfers were excluded), not problematic.

**Investigate PARID = "09F140000803501" for 2017 where GRANTOR = DR HORTON-WPH LLC and GRANTOR_match = NA**

In [40]:
check_parid("09F140000803501")

Sales data with match info


,TAXYR,PARID,Saledt,GRANTEE,GRANTOR,GRANTOR_match,GRANTOR_exact,GRANTOR_single_sale,GRANTOR_only_exact_name,GRANTOR_match_addr,SALES PRICE
43007,2017,09F140000803501,10-NOV-2016,HART RONALD L,DR HORTON-WPH LLC,<NA>,<NA>,<NA>,<NA>,<NA>,145645.00000
88720,2021,09F140000803501,11-MAY-2020,WEEKES GAIL,SPH PROPERTY TWO LLC A DELAWARE LIMITE,HART RONALD L,<NA>,HART RONALD L,<NA>,6352 WOODWELL 30291,170000.00000



Digest data with parcel info


,PARID,TAXYR,Own1,Own2,owner_addr
167775,09F140000803501,2017,HART RONALD L,,6352 WOODWELL 30291
167776,09F140000803501,2018,HART RONALD L,,6352 WOODWELL 30291
167777,09F140000803501,2019,HART RONALD L,,6352 WOODWELL 30291
167778,09F140000803501,2020,HART RONALD L,,6352 WOODWELL 30291
167779,09F140000803501,2021,WEEKES GAIL,,6352 WOODWELL 30291
167780,09F140000803501,2022,WEEKES GAIL,,6352 WOODWELL 30291


Parcel did not exist in records before 2017 and the company name could not be matched with any other methods. This is fine, we don't want false positives.

**Investigate PARID = "14 015200100104" for 2012 where GRANTOR = CSF ENTERPRISES LLC and GRANTOR_match = CPI HOUSING FUND LLC; and 2015 where GRANTOR = ELKINS INVESTMENT LLC and GRANTOR_match = NA**

In [41]:
check_parid("14 015200100104")

Sales data with match info


,TAXYR,PARID,Saledt,GRANTEE,GRANTOR,GRANTOR_match,GRANTOR_exact,GRANTOR_single_sale,GRANTOR_only_exact_name,GRANTOR_match_addr,SALES PRICE
7671,2012,14 015200100104,14-APR-2011,ELKINS INVESTMENTS LLC,CSF ENTERPRISES LLC,CSF ENTERPRISES LLC,<NA>,<NA>,CSF ENTERPRISES LLC,212 16TH 30363,15500.00000
33873,2015,14 015200100104,21-JAN-2014,RHA 1 LLC,ELKINS INVESTMENT LLC,<NA>,<NA>,<NA>,<NA>,<NA>,34000.00000



Digest data with parcel info


,PARID,TAXYR,Own1,Own2,owner_addr
1817006,14 015200100104,2010,BABATOPE DAVID A,,0 PO BOX 747 30168
1817004,14 015200100104,2015,RHA 1 LLC,,3505 KOGER 30096
1817016,14 015200100104,2016,RHA 1 LLC,,3505 KOGER 30096
1817005,14 015200100104,2017,RHA 1 LLC,,3505 KOGER 30096
1817011,14 015200100104,2018,RHA 1 LLC,,3505 KOGER 30096
1817012,14 015200100104,2019,FYR SFR BORROWER LLC,,5100 TAMARIND REEF 820
1817013,14 015200100104,2020,FYR SFR BORROWER LLC,,3505 KOGER BLVD 30096
1817014,14 015200100104,2021,FYR SFR BORROWER LLC,,3505 KOGER 30096
1817015,14 015200100104,2022,FYR SFR BORROWER LLC,,3505 KOGER 30096


Looks like CSF Enterprises and CPI Housing Fund are related and the former carried out the transcation for the latter. We are also missing some parcel records for 2012-2014 (see below if this is a common issue)

**Roughly many parcels are missing an entry during a year?**

In [42]:
count_of_records_parcel = pd.DataFrame(digest_full_geo_nbhd.groupby(by="PARID")["PARID"].count()).rename(columns={"PARID": "count_records"})
print(f"Total number of parcels: {len(count_of_records_parcel)}")
print(f"Number of parcels that don't have a record for every year in period: {len(count_of_records_parcel[count_of_records_parcel["count_records"] < 13])}")
print("")

tot_parcels_begin = digest_full_geo_nbhd[digest_full_geo_nbhd["TAXYR"] == 2010].merge(
    count_of_records_parcel,
    on="PARID",
    how="inner"
)

parcels_missing_some = tot_parcels_begin[tot_parcels_begin["count_records"] < 13]
tot_count_missing = 13 * tot_parcels_begin.count() - tot_parcels_begin[tot_parcels_begin["count_records"] < 13]["count_records"].sum()
print(f"Total number of parcels around at start of study period: {len(tot_parcels_begin)}")
print(f"Number of parcels around at start of period that don't have any entry for every year: {len(parcels_missing_some)}")
print(f"Total number of missing entries: {tot_count_missing}")

Total number of parcels: 226986
Number of parcels that don't have a record for every year in period: 55174

Total number of parcels around at start of study period: 200925
Number of parcels around at start of period that don't have any entry for every year: 29113
Total number of missing entries: PARID             2309939
OBJECTID          2309939
geometry          2309939
TAXYR             2309939
Situs Adrno       2309939
Situs Adrdir      2309939
Situs Adrstr      2309939
Situs Adrsuf      2309939
Cityname          2309939
Luc               2309939
Calcacres         2309939
Own1              2309939
Own2              2309939
Owner Adrno       2309939
Owner Adradd      2309939
Owner Adrdir      2309939
Owner Adrstr      2309939
Owner Adrsuf      2309939
own_cityname      2309939
Statecode         2309939
own_zip           2309939
D Yrblt           2309939
D Effyr           2309939
D Yrremod         2309939
Sfla              2309939
neighborhood       655481
mod_own_adrstr    2309939
o

A fair portion of parcels don't have records for every year, but most of those are parcels which were not around during the beginning of the period. They were likely created during the study period. For parcels that were around at the beginning, ~15% of parcels are missing at least one record.

Validate entire process is working as expected with a sample

In [43]:
fulton_sales_all.sample(20)[
    [
        "TAXYR", "PARID", "GRANTEE", "GRANTEE_match",
        "GRANTOR", "GRANTOR_match"
    ]
].sort_values(by="TAXYR")

,TAXYR,PARID,GRANTEE,GRANTEE_match,GRANTOR,GRANTOR_match
10824,2013,09F290001191198,EDWARDS PAULA,EDWARDS PAULA,STONEVIEW CAPITAL LLC,STONEVIEW CAPITAL LLC
17330,2013,22 464011050913,WILSON KEVIN S,WILSON KEVIN S &,WOODARD JR LAWRENCE,WOODARD LAWRENCE & MELANIE A
20445,2014,12 206204710261,LOHMEYER ASHLEY M,LOHMEYER ASHLEY M,KELL JOSEPH M,KELL JOSEPH M & KAREN J
47513,2017,14 006200010146,LINDSEY LEGACY LLC,LINDSEY LEGACY LLC,NAIROBI INVESTMENT PROPERTIES LLC,NAIROBI INVESTMENT PR LLC
44416,2017,11 087403150570,HP GEORGIA I LLC,HPA BORROWER 2016 2 LLC,DALEY JAMES & SARAH M,DALEY JAMES &
56977,2018,12 212004930463,RAINONE DAVID & JANET,RAINONE DAVID & JANET,MCGOWAN ROBERT K & SWARTZ CHARLENE R,MCGOWAN ROBERT K &
65951,2019,09F370001554247,HAMBRICK DORISHA D,HAMBRICK DORISHA D,SILVERSTONE RESIDENTIAL AT SOMERSET LLC,SILVERSTONE RESIDENTIAL AT SOMERSET LLC
71831,2019,17 000100110083,GONZALEZ OLAYA DANIEL A &,GONZALEZ OLAYA DANIEL A &,CUMMINGS JASON THOMAS,CUMMINGS JASON THOMAS &
73619,2019,17 019400011098,BEDELL LINDSEY W & DANIEL B,BEDELL LINDSEY W & DANIEL B,MITCHELL JOHN L JR,MITCHELL JOHN L JR
70317,2019,14 013800021432,KHATTAB MUDASSAR,KHATTAB MUDASSAR,PETERSON VICTORIA ALEXANDRIA,SMITH QUINCY


### Determine scale of buying and selling activity of each entity for each transcaction in year of the transcaction; create a transaction scale table.
For each GRANTEE (buyer) and GRANTOR (seller), we want a table showing that entity and their number of sales in each year, their number of purchases in each year, and total number of transactions (derived from previous two), at the neighborhood, city, and county level.

Steps:
- Add neighborhoods to sale data
- Aggreggate the number of times a GRANTEE_match_addr shows up in the data for each year. This is the count of purchases for all Fulton.
- Drop non-Atlanta sales; agg the number of times GRANTEE_match_addr shows up in the data for each year. This is the count of purchases for all Atlanta.
- From Atalnta sales; agg the number of times GRANTEE_match-addr shows up for each year, neighborhood combination. This is the count of purchases by neighborhood.
- Repeat the previous 3 steps for GRANTOR.
- To create a tally over the entire study period: agg over TAXYR

In [44]:
# Add neighborhoods to sales data
fulton_sales_all = fulton_sales_all.merge(
    digest_full_geo_nbhd[["PARID", "TAXYR", "neighborhood"]],
    on=["PARID", "TAXYR"],
    how="left"
)
len(fulton_sales_all)

113553

In [81]:
# Agg number of times entity has bought and sold in the data for each year
# ----------------------------------------------------------------------------------------------
df = fulton_sales_all.copy(deep=True)
trans_levels = []

for level in ["fulton", "atl", "neighborhood"]:
    
    # Take only ATL sales
    if level in ["atl", "neighborhood"]:
        df = df[df["neighborhood"].notna()]
        
    if level in "neighborhood":
        count_sales_yr = pd.DataFrame(
            df.groupby(["TAXYR", "neighborhood"])["PARID"].count().reset_index().rename(
                columns={"PARID": f"count_{level}_sales"}
            )
        )

        entity_count_purchases = df.groupby(["TAXYR", "GRANTEE_match_addr", "neighborhood"]).agg(
            {"GRANTEE_match_addr": "count"}
        ).rename(
            columns={"GRANTEE_match_addr": f"entity_count_purchases_{level}"}
        ).reset_index()

        entity_count_sales = df.groupby(["TAXYR", "GRANTOR_match_addr", "neighborhood"]).agg(
            {"GRANTOR_match_addr": "count"}
        ).rename(
            columns={"GRANTOR_match_addr": f"entity_count_sales_{level}"}
        ).reset_index()
        
        entity_count_trans = entity_count_purchases.merge(
            entity_count_sales,
            left_on=["TAXYR", "GRANTEE_match_addr", "neighborhood"],
            right_on=["TAXYR", "GRANTOR_match_addr", "neighborhood"],
            how="outer"
        ).merge(
                count_sales_yr,
                on=["TAXYR", "neighborhood"],
                how="inner"
        )
        
        entity_count_trans["GRANTEE_match_addr"] = entity_count_trans["GRANTEE_match_addr"].fillna(
            entity_count_trans["GRANTOR_match_addr"]
        )
        
        entity_count_trans = entity_count_trans.rename(columns={
            "GRANTEE_match_addr": "entity_addr"
        }).drop(columns=["GRANTOR_match_addr"])
        
        trans_levels.append(entity_count_trans)

    else:
        count_sales_yr = pd.DataFrame(
            df.groupby("TAXYR")["PARID"].count().reset_index().rename(
                columns={"PARID": f"count_{level}_sales"}
            )
        )

        entity_count_purchases = df.groupby(["TAXYR", "GRANTEE_match_addr"]).agg(
            {"GRANTEE_match_addr": "count"}
        ).rename(
            columns={"GRANTEE_match_addr": f"entity_count_purchases_{level}"}
        ).reset_index()

        entity_count_sales = df.groupby(["TAXYR", "GRANTOR_match_addr"]).agg(
            {"GRANTOR_match_addr": "count"}
        ).rename(
            columns={"GRANTOR_match_addr": f"entity_count_sales_{level}"}
        ).reset_index()

        entity_count_trans = entity_count_purchases.merge(
            entity_count_sales,
            left_on=["TAXYR", "GRANTEE_match_addr"],
            right_on=["TAXYR", "GRANTOR_match_addr"],
            how="outer"
        ).merge(
                count_sales_yr,
                on="TAXYR",
                how="inner"
        )
        
        entity_count_trans["GRANTEE_match_addr"] = entity_count_trans["GRANTEE_match_addr"].fillna(
            entity_count_trans["GRANTOR_match_addr"]
        )
        
        entity_count_trans = entity_count_trans.rename(columns={
            "GRANTEE_match_addr": "entity_addr"
        }).drop(columns=["GRANTOR_match_addr"])
        
        trans_levels.append(entity_count_trans)

In [90]:
all_trans_scale = trans_levels[0].merge(
    trans_levels[1],
    on=["TAXYR", "entity_addr"],
    how="inner"
).merge(
    trans_levels[2],
    on=["TAXYR", "entity_addr"],
    how="inner"
)

# To create a tally over the entire study period: agg over TAXYR
all_trans_scale["total_trans"] = all_trans_scale
all_trans_scale.sort_values(by="entity_count_purchases_fulton", ascending=False).sample(5)

,TAXYR,entity_addr,entity_count_purchases_fulton,entity_count_sales_fulton,count_fulton_sales,entity_count_purchases_atl,entity_count_sales_atl,count_atl_sales,neighborhood,entity_count_purchases_neighborhood,entity_count_sales_neighborhood,count_neighborhood_sales
39579,2020,6420 BERKDALE 30024,1.00000,NaN,12278,1.00000,NaN,3884,"Blair Villa/Poole Creek, Glenrose Heights, Orc...",1.00000,NaN,32
29891,2018,840 BOLTON 30331,1.00000,NaN,10613,1.00000,NaN,3065,"Bankhead Courts, Bankhead/Bolton, Carroll Heig...",1.00000,NaN,12
43436,2021,1911 LA DAWN 30318,1.00000,NaN,12645,1.00000,NaN,4183,"Bolton, Riverside, Whittier Mill Village",1.00000,NaN,131
13969,2014,4514 OAK BROOK 30082,NaN,1.00000,10549,NaN,1.00000,3863,"Benteen Park, Boulevard Heights, Custer/McDono...",NaN,1.00000,71
17349,2015,709 GLENWOOD 30312,1.00000,1.00000,10217,1.00000,1.00000,3660,"Grant Park, Oakland",1.00000,1.00000,165


In [ ]:
# TODO check Fulton percent demon is correct (should be by year)
# one sale has both a buyer and seller, how does this impact denom
all_trans_levels = {}

for i, person in enumerate(["GRANTEE", "GRANTOR"]):
    for level in ["fulton", "atl", "neighborhood"]:
        if level == "fulton":
            fulton_sale_count_yr = pd.DataFrame(
                fulton_sales_all.groupby("TAXYR")["PARID"].count()
            ).rename(columns={
                "PARID": "fulton_sales_taxyr"
            })
            
            sales_fulton_by_entity = fulton_sales_all.groupby(
                ["TAXYR", f"{person}_match_addr"]
            ).agg(
                {f"{person}_match_addr": "count"}
            ).rename(
                columns={f"{person}_match_addr": f"{person}_count_fulton"}
            ).reset_index().merge(
                fulton_sale_count_yr,
                on="TAXYR",
                how="inner"
            )
            
            sales_fulton_by_entity[f"pct_{person}_fulton"] = sales_fulton_by_entity[
                f"{person}_count_fulton"
            ] / sales_fulton_by_entity[
                "fulton_sales_taxyr"
            ] * 100
            
        elif level == "atl":
            atl_parcels_list = digest_full_geo_nbhd[
                digest_full_geo_nbhd["neighborhood"].notna()
            ]["PARID"].unique().tolist()
            
            atl_sales = fulton_sales_all[fulton_sales_all["PARID"].isin(atl_parcels_list)]
            
            atl_sale_count_yr = pd.DataFrame(
                atl_sales.groupby("TAXYR")["PARID"].count()
            ).rename(columns={
                "PARID": "atl_sales_taxyr"
            })
            
            sales_atl_by_entity = atl_sales.groupby(
                ["TAXYR", f"{person}_match_addr"]
            ).agg(
                {f"{person}_match_addr": "count"}
            ).rename(
                columns={f"{person}_match_addr": f"{person}_count_atl"}
            ).reset_index().merge(
                atl_sale_count_yr,
                on="TAXYR",
                how="inner"
            )
            
            sales_atl_by_entity[f"pct_{person}_atl"] = sales_atl_by_entity[
                f"{person}_count_atl"
            ] / sales_atl_by_entity[
                "atl_sales_taxyr"
            ] * 100
            
        else:
            nbhd_sale_count_yr = pd.DataFrame(
                atl_sales.groupby(
                    ["neighborhood", "TAXYR"]
                )["PARID"].count()
            ).rename(columns={"PARID": "neighborhood_sales_taxyr"})

            sales_neighborhood_by_entity = atl_sales.groupby(
                ["TAXYR", f"{person}_match_addr", "neighborhood"]
            ).agg(
                {f"{person}_match_addr": "count"}
            ).rename(
                columns={f"{person}_match_addr": f"{person}_count_neighborhood"}
            ).reset_index().merge(
                nbhd_sale_count_yr,
                on=["TAXYR", "neighborhood"],
                how="inner"
            )

            sales_neighborhood_by_entity[f"pct_{person}_neighborhood"] = sales_neighborhood_by_entity[
                f"{person}_count_neighborhood"
            ] / sales_neighborhood_by_entity[
                "neighborhood_sales_taxyr"
            ] * 100
            
    all_trans_levels[f"{person}"] = sales_fulton_by_entity.merge(
        sales_atl_by_entity, on=["TAXYR", f"{person}_match_addr"], how="inner"
    ).merge(
        sales_neighborhood_by_entity, on=["TAXYR", f"{person}_match_addr"], how="outer"
    )

# all_trans_levels[1].merge(
#     all_trans_levels[2],
#     on=["TAXYR", "owner_addr"],
# )

# all sales in Fulton, we can make a count column

# all purchases in Fulton, make count

# sum counts for sale and purchase


In [ ]:
all_trans_levels["GRANTEE"].sample(3, random_state=10)

,TAXYR,GRANTEE_match_addr,GRANTEE_count_fulton,fulton_sales_taxyr,pct_GRANTEE_fulton,GRANTEE_count_atl,atl_sales_taxyr,pct_GRANTEE_atl,neighborhood,GRANTEE_count_neighborhood,neighborhood_sales_taxyr,pct_GRANTEE_neighborhood
7469,2014,2300 LAS OLAS 33301,157,10549,1.48829,148,3983,3.71579,"Hunter Hills, Mozley Park",11.00000,59.00000,18.64407
15594,2017,2487 FERNLEAF 30318,1,11498,0.00870,1,3827,0.02613,"Fernleaf, Hanover West, Ridgewood Heights, Wes...",1.00000,39.00000,2.56410
28452,2021,1360 9TH 44114,1,12645,0.00791,1,4206,0.02378,"Chastain Park, Tuxedo Park",1.00000,57.00000,1.75439


In [ ]:
# Validate first entry for GRANTEE count fulton, count atl, count neighborhood
print(fulton_sales_all[
    (fulton_sales_all["GRANTEE_match_addr"] == "2300 LAS OLAS 33301") &
    (fulton_sales_all["TAXYR"] == 2014)
]["PARID"].count())

print(fulton_sales_all[
    (fulton_sales_all["neighborhood"].notna()) &
    (fulton_sales_all["GRANTEE_match_addr"] == "2300 LAS OLAS 33301") &
    (fulton_sales_all["TAXYR"] == 2014)
]["PARID"].count())

print(fulton_sales_all[
    (fulton_sales_all["neighborhood"] == "Hunter Hills, Mozley Park") &
    (fulton_sales_all["GRANTEE_match_addr"] == "2300 LAS OLAS 33301") &
    (fulton_sales_all["TAXYR"] == 2014)
]["PARID"].count())

# Looks good

157
148
11


In [ ]:
all_trans_levels["GRANTOR"].sample(3, random_state=10)

,TAXYR,GRANTOR_match_addr,GRANTOR_count_fulton,fulton_sales_taxyr,pct_GRANTOR_fulton,GRANTOR_count_atl,atl_sales_taxyr,pct_GRANTOR_atl,neighborhood,GRANTOR_count_neighborhood,neighborhood_sales_taxyr,pct_GRANTOR_neighborhood
29953,2021,833 KIPLING 30318,1,12645,0.00791,1,4206,0.02378,"Channing Valley, Memorial Park, Springlake, Wi...",1.00000,79.00000,1.26582
4319,2013,207 WINNONA 30030,6,7495,0.08005,6,2600,0.23077,"Hunter Hills, Mozley Park",1.00000,33.00000,3.03030
32047,2022,2534 DELLWOOD 30305,1,13211,0.00757,1,4649,0.02151,"Arden/Habersham, Argonne Forest, Peachtree Bat...",1.00000,67.00000,1.49254


In [ ]:
# Validate first entry for GRANTOR count fulton, count atl, count neighborhood
print(fulton_sales_all[
    (fulton_sales_all["GRANTOR_match_addr"] == "207 WINNONA 30030") &
    (fulton_sales_all["TAXYR"] == 2013)
]["PARID"].count())

print(fulton_sales_all[
    (fulton_sales_all["neighborhood"].notna()) &
    (fulton_sales_all["GRANTOR_match_addr"] == "207 WINNONA 30030") &
    (fulton_sales_all["TAXYR"] == 2013)
]["PARID"].count())

print(fulton_sales_all[
    (fulton_sales_all["neighborhood"] == "Hunter Hills, Mozley Park") &
    (fulton_sales_all["GRANTOR_match_addr"] == "207 WINNONA 30030") &
    (fulton_sales_all["TAXYR"] == 2013)
]["PARID"].count())

# Looks good

6
6
1


In [ ]:
len(fulton_sales_all)

113553

In [ ]:
all_trans_levels["GRANTEE"][all_trans_levels["GRANTEE"].duplicated(subset=["TAXYR", "GRANTEE_match_addr"])]

,TAXYR,GRANTEE_match_addr,GRANTEE_count_fulton,fulton_sales_taxyr,pct_GRANTEE_fulton,GRANTEE_count_atl,atl_sales_taxyr,pct_GRANTEE_atl,neighborhood,GRANTEE_count_neighborhood,neighborhood_sales_taxyr,pct_GRANTEE_neighborhood
14,2011,0 PO BOX 19662 30325,3,5459,0.05496,3,1975,0.15190,"Adair Park, Pittsburgh",1.00000,42.00000,2.38095
15,2011,0 PO BOX 19662 30325,3,5459,0.05496,3,1975,0.15190,"Hunter Hills, Mozley Park",1.00000,36.00000,2.77778
17,2011,0 PO BOX 1996 29063,3,5459,0.05496,3,1975,0.15190,"Adair Park, Pittsburgh",1.00000,42.00000,2.38095
18,2011,0 PO BOX 1996 29063,3,5459,0.05496,3,1975,0.15190,"Capitol View, Capitol View Manor",1.00000,22.00000,4.54545
25,2011,0 PO BOX 366512 30336,6,5459,0.10991,6,1975,0.30380,West End,1.00000,25.00000,4.00000
...,...,...,...,...,...,...,...,...,...,...,...,...
36377,2022,9200 HAMPTON 20743,71,13211,0.53743,31,4649,0.66681,"Fort McPherson, Venetian Hills",2.00000,85.00000,2.35294
36378,2022,9200 HAMPTON 20743,71,13211,0.53743,31,4649,0.66681,"Adamsville, Oakcliff",1.00000,25.00000,4.00000
36379,2022,9200 HAMPTON 20743,71,13211,0.53743,31,4649,0.66681,Ivan Hill,2.00000,15.00000,13.33333
36472,2022,950 EAGLES LANDING 30281,2,13211,0.01514,2,4649,0.04302,Westview,1.00000,83.00000,1.20482


In [ ]:
len(all_trans_levels["GRANTEE"])

36616

In [ ]:
len(all_trans_levels["GRANTOR"])

34531

I need to calculate the number of purchases (e.g. where GRANTEE) and number of sales (e.g. where GRANTOR) for each owner in Fulton, in ATL, and in each neighborhood by year

Then create a neighborhood table for total number of trans in a neighborhood for TAXYR then merge
Then make a new column for total number in TAXYR from Fulton, merge
Total number in TAXYR in ATL, merge

In [ ]:
all_trans_levels["GRANTEE"].merge(
    all_trans_levels["GRANTOR"],
    left_on=["TAXYR", "GRANTEE_match_addr", "neighborhood"],
    right_on=["TAXYR", "GRANTOR_match_addr", "neighborhood"],
    how="outer"
)[["neighborhood", "GRANTEE_count_neighborhood", "pct_GRANTEE_neighborhood",
   "GRANTOR_count_neighborhood", "pct_GRANTEE_neighborhood",
   "neighborhood_sales_taxyr_x", "neighborhood_sales_taxyr_y"]].sample(5)

,neighborhood,GRANTEE_count_neighborhood,pct_GRANTEE_neighborhood,GRANTOR_count_neighborhood,pct_GRANTEE_neighborhood,neighborhood_sales_taxyr_x,neighborhood_sales_taxyr_y
53599,"Atlanta Industrial Park, Bolton Hills, Brookvi...",NaN,NaN,2.00000,NaN,NaN,36.00000
42354,Morningside/Lenox Park,NaN,NaN,1.00000,NaN,NaN,149.00000
13682,Piedmont Heights,1.00000,4.16667,1.00000,4.16667,24.00000,24.00000
1093,"Ben Hill Terrace, Kings Forest, Old Fairburn V...",1.00000,50.00000,1.00000,50.00000,2.00000,2.00000
9446,Ormewood Park,1.00000,0.99010,NaN,0.99010,101.00000,NaN


In [ ]:
# Merge back to sales data
fulton_sales_all = fulton_sales_all.merge(
    all_trans_levels["GRANTEE"],
    on=["TAXYR", "GRANTEE_match_addr", "neighborhood"],
    how="left"
).merge(
    all_trans_levels["GRANTOR"],
    on=["TAXYR", "GRANTOR_match_addr", "neighborhood"],
    how="left"
)

#fulton_sales_all["total_"] = fulton_sales_all[""]

,TAXYR,PARID,Luc,Saledt,SALES PRICE,FAIR MARKET VALUE,DEED TYPE,Costval,Saleval,GRANTOR,GRANTEE,count_sales_yr,GRANTEE_exact,GRANTEE_exact_addr,GRANTEE_single_sale,GRANTEE_single_sale_addr,GRANTEE_only_exact_name,GRANTEE_only_exact_name_addr,GRANTOR_exact,GRANTOR_exact_addr,GRANTOR_single_sale,GRANTOR_single_sale_addr,GRANTOR_only_exact_name,GRANTOR_only_exact_name_addr,GRANTEE_match,GRANTEE_match_addr,GRANTOR_match,GRANTOR_match_addr,neighborhood,GRANTEE_count_fulton,fulton_sales_taxyr_x,pct_GRANTEE_fulton,GRANTEE_count_atl,atl_sales_taxyr_x,pct_GRANTEE_atl,GRANTEE_count_neighborhood,neighborhood_sales_taxyr_x,pct_GRANTEE_neighborhood,GRANTOR_count_fulton,fulton_sales_taxyr_y,pct_GRANTOR_fulton,GRANTOR_count_atl,atl_sales_taxyr_y,pct_GRANTOR_atl,GRANTOR_count_neighborhood,neighborhood_sales_taxyr_y,pct_GRANTOR_neighborhood
0,2011,06 0310 LL0490,101,07-JUN-2010,794600.00000,717100.00000,WD,717100,0,CDG HOMES LLC,EDMUNDS KEITH S & KIMBERLY C,1,EDMUNDS KEITH S & KIMBERLY C,3916 DAHLWINY 30350,EDMUNDS KEITH S & KIMBERLY C,3916 DAHLWINY 30350,EDMUNDS KEITH S & KIMBERLY C,3916 DAHLWINY 30350,<NA>,<NA>,<NA>,<NA>,CDG HOMES LLC,1235 HIGHTOWER 30350,EDMUNDS KEITH S & KIMBERLY C,3916 DAHLWINY 30350,CDG HOMES LLC,1235 HIGHTOWER 30350,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2011,06 0310 LL0581,101,14-JUL-2010,800000.00000,590400.00000,WD,590400,0,CAPITAL DESIGN HOMES LLC,MEHDIPOUR MOHAMMADREZ & SADEGHI SHIVA,1,<NA>,<NA>,MEHDIPOUR MOHAMMADREZA &,3952 DAHLWINY 30350,<NA>,<NA>,<NA>,<NA>,CDG HOMES LLC,1235 HIGHTOWER 30350,CAPITAL DESIGN HOMES LLC,1235 HIGHTOWER 30327,MEHDIPOUR MOHAMMADREZA &,3952 DAHLWINY 30350,CDG HOMES LLC,1235 HIGHTOWER 30350,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2011,06 031000020232,101,28-JAN-2010,437500.00000,437500.00000,WD,465100,0,STUMP BLAIR E,DEMPSEY JASON,1,DEMPSEY JASON,340 ALDENSHIRE 30350,DEMPSEY JASON,340 ALDENSHIRE 30350,DEMPSEY JASON,340 ALDENSHIRE 30350,<NA>,<NA>,STUMP BLAIR E & KAYE A,340 ALDENSHIRE 30350,<NA>,<NA>,DEMPSEY JASON,340 ALDENSHIRE 30350,STUMP BLAIR E & KAYE A,340 ALDENSHIRE 30350,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2011,06 031100030032,101,13-AUG-2010,475000.00000,451300.00000,WD,451300,0,MIDDLETON ERIC C & SUZUKI MASAMI A,BROWN EDWARD H,1,<NA>,<NA>,BROWN EDWARD H & DEBRA Y,150 SAVANNAH ESTATES 30350,<NA>,<NA>,<NA>,<NA>,SUZUKI MASAMI A &,150 SAVANNAH ESTATES 30350,<NA>,<NA>,BROWN EDWARD H & DEBRA Y,150 SAVANNAH ESTATES 30350,SUZUKI MASAMI A &,150 SAVANNAH ESTATES 30350,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2011,06 031100050022,101,12-MAR-2010,497500.00000,497500.00000,WD,504600,0,GATHERS JEFFREY L & DONNA M,PATTERSON BILLY M JR & MELISSA C,1,PATTERSON BILLY M JR & MELISSA C,405 WOODALL 30350,PATTERSON BILLY M JR & MELISSA C,405 WOODALL 30350,PATTERSON BILLY M JR & MELISSA C,405 WOODALL 30350,GATHERS JEFFREY L & DONNA M,405 WOODALL 30350,GATHERS JEFFREY L & DONNA M,405 WOODALL 30350,GATHERS JEFFREY L & DONNA M,405 WOODALL 30350,PATTERSON BILLY M JR & MELISSA C,405 WOODALL 30350,GATHERS JEFFREY L & DONNA M,405 WOODALL 30350,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113548,2022,22 542009742045,101,08-NOV-2021,385500.00000,349200.00000,LW,"349,200",0,OPENDOOR PROPERTY J LLC,WALKER KIM TERESA,1,WALKER KIM TERESA,5530 HIGHLAND PRESERVE 30126,WALKER KIM TERESA,5530 HIGHLAND PRESERVE 30126,WALKER KIM TERESA,5530 HIGHLAND PRESERVE 30126,<NA>,<NA>,SIVAMANI RADHAKANNAN,3161 AVENSONG VILLAGE 30004,OPENDOOR PROPERTY J LLC,405 HOWARD 94105,WALKER KIM TERESA,5530 HIGHLAND PRESERVE 30126,SIVAMANI RADHAKANNAN,3161 AVENSONG VILLAGE 30004,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
113549,2022,22

In [ ]:
# Save output
fulton_sales_all.to_csv("./output/fulton_sales_owner_matches.csv", index=False)

### Identify corporate owners, create corp owner flags for each record (grantee, grantor, and own1 in sales and digest);

### For each sale, create a dummy variable for each sale type: corp purchase from ind, ind purchase from ind, corp sale to ind, ind sale from ind (should be identical to other ind to ind metric)
Flags:
- One for any corp owner
- One for corps who bought from ind after the study period began (depends on sale type matrix)

In [ ]:
# Any with risk of false positive like "CO" need to have a space prepended or postpended
corp_keywords = [
    'LLC', ' INC', 'LLP', 'L.L.C', 'L.L.P', 'I.N.C', 'L L C',
    'L L P', ' L P', 'LTD', ' CORP', 'CORPORATION',
    'COMPANY', ' CO ', 'LIMITED', 'PARTNERSHIP', 'PARTNERSHIPS',
    'ASSOCIATION', 'ASSOC', 'INCORPORATED', 'INCORP',
    'L.T.D', 'LTD', "HOME"
]

# Make a list of all corp owners
corps = fulton_sales_all[
    fulton_sales_all['GRANTEE'].apply(lambda x: any([key in str(x) for key in corp_keywords]))
]['GRANTEE'].unique().tolist() + fulton_sales_all[
    fulton_sales_all['GRANTOR'].apply(lambda x: any([key in str(x) for key in corp_keywords]))
]['GRANTOR'].unique().tolist() + digest_full_geo_nbhd[
    digest_full_geo_nbhd["Own1"].apply(lambda x: any([key in str(x) for key in corp_keywords]))
]['Own1'].unique().tolist()

with open("./output/corp_names.txt", "w") as f:
    f.write("\n".join(corps))

In [ ]:
# Flag for any corp owner
fulton_sales_all["GRANTEE_corp_flag"] = fulton_sales_all['GRANTEE'].isin(corps).astype(int)
fulton_sales_all["GRANTOR_corp_flag"] = fulton_sales_all['GRANTOR'].isin(corps).astype(int)

digest_full_geo_nbhd["own_corp_flag"] = digest_full_geo_nbhd["Own1"].isin(corps).astype(int)

# Sale type matrix

fulton_sales_all['corp_bought_ind'] = 0
fulton_sales_all['corp_sold_ind'] = 0
fulton_sales_all['ind_to_ind'] = 0
fulton_sales_all['corp_to_corp'] = 0

fulton_sales_all.loc[
    (fulton_sales_all["GRANTEE_corp_flag"] == 1) & (fulton_sales_all["GRANTOR_corp_flag"] == 0), 'corp_bought_ind'
] = 1
fulton_sales_all.loc[
    (fulton_sales_all["GRANTEE_corp_flag"] == 0) & (fulton_sales_all["GRANTOR_corp_flag"] == 0), 'ind_to_ind'
] = 1
fulton_sales_all.loc[
    (fulton_sales_all["GRANTEE_corp_flag"] == 0) & (fulton_sales_all["GRANTOR_corp_flag"] == 1), 'corp_sold_ind'
] = 1
fulton_sales_all.loc[
    (fulton_sales_all["GRANTEE_corp_flag"] == 1) & (fulton_sales_all["GRANTOR_corp_flag"] == 1), 'corp_to_corp'
] = 1

# Validate sale matrix is correct
fulton_sales_all[[
    "GRANTEE", "GRANTEE_corp_flag", "GRANTOR", "GRANTOR_corp_flag", "corp_bought_ind", "ind_bought_ind",
    "corp_sold_ind", "ind_sold_ind"
]].sample(10)

,GRANTEE,GRANTEE_corp_flag,GRANTOR,GRANTOR_corp_flag,corp_bought_ind,ind_bought_ind,corp_sold_ind,ind_sold_ind
93181,NIXON TIFFANY TIERA,0,ATLANTA NEIGHBORHOOD AND DEVELOPMENT PAR,0,0,1,0,1
40943,BOYD DAVID E,0,SECORD LORENE A,0,0,1,0,1
10792,THR GEORGIA LP,0,TRIBBLE DEBORA,0,0,1,0,1
39830,VAN GELDER PHILIP & MORGAN,0,HANKINS RICHARD B & MELISSA A,0,0,1,0,1
101256,BENOIT THERESSA,0,CORNERSTONE FULTON HOME BUILDERS INC,1,0,0,1,0
94141,KEEN OBAID IQBAL ET AL,0,SORAK MARK,0,0,1,0,1
86880,MOHAN KELLEN & LAURA,0,PADEN ASHLEY JONES,0,0,1,0,1
59627,DAVIS JASON JON,0,ARAIM MANAGEMENT 1 LLC,1,0,0,1,0
6093,BURTCHAELL RHONDA C,0,HILLS LAURENCE G,0,0,1,0,1
71172,DIXON GWENDOLYN CROCKETT,0,SHELLI BARNES N/K/A SHELLI BARNES DAVIS,0,0,1,0,1


In [ ]:
# Flag for corp owner bought after study period began
parcels_corp_from_ind = set(fulton_sales_all[fulton_sales_all["corp_bought_ind"] == 1]['PARID'].unique())
digest_full_geo_nbhd["corp_bought_after_2010"] = digest_full_geo_nbhd["PARID"].apply(lambda x: 1 if x in parcels_corp_from_ind else 0)

# Validate
digest_full_geo_nbhd[["TAXYR", "PARID", "corp_bought_after_2010"]].sample(10)

,TAXYR,PARID,corp_bought_after_2010
1222604,2019,17 010500040335,0
2596007,2017,09F400201620918,0
1444166,2010,17 021400010187,0
35649,2013,22 495110380199,0
1946428,2010,14 021700040394,0
1403006,2015,17 009800010238,0
1104274,2018,17 014600050608,0
690241,2015,12 243205790182,0
1640448,2010,14 011500030059,0
679698,2014,12 303108420046,0


In [ ]:
# Validate continued; only last should have been bought by a corp from ind
display(fulton_sales_all[fulton_sales_all["PARID"] == "14 001100110161"][["TAXYR", "PARID", "GRANTEE", "GRANTOR", "corp_bought_ind"]])
display(fulton_sales_all[fulton_sales_all["PARID"] == "17 000200100547"][["TAXYR", "PARID", "GRANTEE", "GRANTOR", "corp_bought_ind"]])
display(fulton_sales_all[fulton_sales_all["PARID"] == "14 017500110080"][["TAXYR", "PARID", "GRANTEE", "GRANTOR", "corp_bought_ind"]])
display(fulton_sales_all[fulton_sales_all["PARID"] == "12 267306750308"][["TAXYR", "PARID", "GRANTEE", "GRANTOR", "corp_bought_ind"]])

,TAXYR,PARID,GRANTEE,GRANTOR,corp_bought_ind


,TAXYR,PARID,GRANTEE,GRANTOR,corp_bought_ind
34952,2015,17 000200100547,JOYE CHARLES M &,KHAJAVI KAVEH,0


,TAXYR,PARID,GRANTEE,GRANTOR,corp_bought_ind


,TAXYR,PARID,GRANTEE,GRANTOR,corp_bought_ind
31318,2015,12 267306750308,SRP SUB LLC,MAJID ABAZERI AND ANA L CRUZ,1


### Understand distribution of corporate ownership size
Within Fulton, within Atlanta, within each neighborhood

In [ ]:
# Continous, 2022 number of properties owned in Fulton by each owner
fig = px.histogram(df, x="total_bill", y="tip", color="sex",
                   marginal="box", # or violin, rug
                   hover_data=df.columns)
fig.show()

In [ ]:
# Bins

### Agg each class of sale

### Get totals for Fulton then drop non-ATL and agg by neighborhoods, year, size of investor

### Track each property after purchase (or at all owned by corp during period), calculate rental income

### Normalized equity loss measure

### Equity loss burden

### Statistical test to see if FMV - SP was significant between ind and corp (ANOVA) or regression

### Create a measure of corp concentration in neighborhood to use as metric for analysis - is it just being a corp that helps, or when there's high concentration?

### Geospatial

### Do neighborhood characteristics predict equity loss

### Foreclosures?